In [17]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import time
import glob
import csv
import os
import sys

from datetime import datetime, timedelta
from tsmoothie.smoother import *
from tsmoothie.utils_func import sim_seasonal_data
from tsmoothie.smoother import DecomposeSmoother
df.

In [19]:
sys.stdout = open('D:\python\log.txt','a')

In [20]:
#today = datetime.now()
today = datetime(2021, 1, 6)
last_date=today-timedelta(weeks=1)
today = today.strftime('%Y-%m-%d')#ex)2019-03-14
last_date = last_date.strftime('%Y-%m-%d')#ex)2019-03-14

In [4]:
import CUBRIDdb
import pandas as pd

conn = CUBRIDdb.connect('CUBRID:100.13.11.209:30000:dgsmwtanalysis:::','dgsmwt','dgwater')
conn.set_autocommit(False)
cur = conn.cursor()

selectQry = """\
    SELECT cstmr_no
    , mrnr_no
    , ok1
    , ok1 - notOk1 AS notOk1
    , ok2
    , ok2 - notOk2 AS notOk2
  FROM (
        SELECT cstmr_no
                 , mrnr_no
                 , CASE WHEN ok = 1 THEN 1
                           WHEN notOk1 = 0 THEN 1
                           ELSE 0
                   END AS ok1
                 , notOk1
                 , CASE WHEN ok = 1 THEN 1
                           WHEN notOk2 = 0 THEN 1
                           ELSE 0
                   END AS ok2
                 , notOk2
         FROM (
                    SELECT cstmr_no
                            , mrnr_no
                            , DECODE(smrt_mtinsp_sttus, '정상', 1, 0) ok
                            , CASE WHEN smrt_mtinsp_sttus = '계량기오류' THEN 1
                                      WHEN smrt_mtinsp_sttus = '계량기모뎀오류' THEN 1
                                      ELSE 0
                              END notOk1
                            , CASE WHEN smrt_mtinsp_sttus = '모뎀오류' THEN 1
                                      WHEN smrt_mtinsp_sttus = '계량기모뎀오류' THEN 1
                                      ELSE 0
                              END notOk2
                    FROM tb_smrt_mtinsp_info
                    WHERE mrnr_no = 'A00'
                        AND mtinsp_dt BETWEEN TO_DATETIME('{last_date} 01:00:00', 'YYYY-MM-DD HH24:MI:SS') AND TO_DATETIME('{today} 00:00:00', 'YYYY-MM-DD HH24:MI:SS')
                  )
        )
""".format(last_date=last_date, today=today)
selectQry = selectQry.replace("\n\t"," ")
selectQry = selectQry.replace("\n"," ") 

cur.execute(selectQry)
rows = cur.fetchall()
df = pd.DataFrame(rows)
df.columns=['cstmr_no', 'mrnr_no', 'ok1', 'notOk1', 'ok2', 'notOk2']

In [5]:
df_du = df.drop_duplicates(["cstmr_no"], keep="last")

In [6]:
try:
    for index, row in df_du.iterrows():
        lst = df[df['cstmr_no'] == row["cstmr_no"]]
        y = lst['ok1']
        y_hat = lst['notOk1']
        RMSE_1 = 1- (np.sqrt(np.mean((y_hat - y)**2))) #계량기오류
        y = lst['ok2']
        y_hat = lst['notOk2']
        RMSE_2 = 1-(np.sqrt(np.mean((y_hat - y)**2))) #모뎀오류
        
        if RMSE_1 <= 0.3 or RMSE_2 <= 0.3:
            insert_query = """\
                INSERT INTO tb_abnrml_anals_test
                (
                    anals_stdr_day,
                    cstmr_no,
                    mrnr_no,
                    mrnr_rmse,
                    trmnl_rmse,
                    frst_regist_dt
                )
                VALUES
                (
                    DATE_FORMAT(SYSDATETIME, '%Y%m%d'),
                    '{cstmr_no}',
                    '{mrnr_no}',
                    {mrnr_rmse},
                    {trmnl_rmse},
                    CURRENT_TIMESTAMP
                )
            """.format(cstmr_no=row['cstmr_no'], mrnr_no=row['mrnr_no'],mrnr_rmse=RMSE_1, trmnl_rmse=RMSE_2)          
            insert_query = insert_query.replace("\n\t"," ")
            insert_query = insert_query.replace("\n"," ")  
            cur.execute(insert_query)
        
        
except Exception as e:
    print("계량기모뎀에러 ::: " + str(datetime.now()))
    print(e)
    conn.rollback()
else:
    conn.commit()